# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [96]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [97]:
# Note: python would only acknowledge the existence of the weather data file if I gave it the whole file address. This has been a persistent problem for me, and I don't know how to resolve it.
weather_data_file = "C:\\Users\\pippi\\OneDrive\\Documents\\nu-chi-data-pt-06-2021-u-c-master\\06-Python-APIs\\Homework\\due_July29\\Instructions\\output_data\\weather_data.csv"
weather_data_original = pd.read_csv(weather_data_file, dtype = "object", encoding = 'utf-8')
weather_data_original.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Nikolskoye,59.7035,30.7861,287.09,71,67,1.07,RU,1628041600
1,1,Mataura,-46.1927,168.8643,275.08,91,100,1.34,NZ,1628107190
2,2,Puerto Ayora,-0.7393,-90.3518,301.13,75,17,3.13,EC,1628078693
3,3,São João da Barra,-21.6403,-41.0511,297.2,65,7,6.77,BR,1628068478
4,4,Meulaboh,4.1363,96.1285,297.63,87,100,1.77,ID,1628119976


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [98]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [100]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data_original[["Lat", "Lng"]].astype(float)
humidity = weather_data_original['Humidity']

# Create the Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                dissipating=False, max_intensity=100,
                                point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [102]:
# Note: Temperature was stored as kelvin instead of farenheit, for reasons that are unknown to me.
weather_data_ideal = weather_data_original.loc[((weather_data_original['Max Temp'].astype(float) > 294.26) & (weather_data_original['Max Temp'].astype(float) < 299.82))]
                                              #(weather_data_original['Wind Speed'].astype(float) < 10.00) | (weather_data_original['Cloudiness'] == 0)
weather_data_idealer = weather_data_ideal.loc[(weather_data_ideal['Wind Speed'].astype(float) < 10.00)]

weather_data_idealest = weather_data_idealer.loc[(weather_data_idealer['Cloudiness'].astype(float) == 0)]
weather_data_idealest.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
80,80,Pellérd,46.0344,18.154,297.62,60,0,2.57,HU,1628047996
138,138,Hasaki,35.7333,140.8333,296.37,89,0,1.63,JP,1628106443
207,207,Emirdağ,39.0197,31.15,296.79,37,0,0.47,TR,1628045921
288,288,Lagoa,39.05,-27.9833,295.83,89,0,5.87,PT,1628060118
311,311,Benguela,-12.5763,13.4055,294.77,85,0,0.61,AO,1628054683


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [103]:
# Store into variable named hotel_df.
hotel_df = pd.DataFrame(weather_data_idealest)
print(hotel_df)

    Unnamed: 0      City       Lat       Lng Max Temp Humidity Cloudiness  \
80          80   Pellérd   46.0344    18.154   297.62       60          0   
138        138    Hasaki   35.7333  140.8333   296.37       89          0   
207        207   Emirdağ   39.0197     31.15   296.79       37          0   
288        288     Lagoa     39.05  -27.9833   295.83       89          0   
311        311  Benguela  -12.5763   13.4055   294.77       85          0   
482        482   Shimoda   34.6667    138.95   295.43       84          0   
484        484  Londrina  -23.3103  -51.1628   294.96       37          0   
493        493   Vysokyy   49.8906     36.12   295.14       46          0   

    Wind Speed Country        Date  
80        2.57      HU  1628047996  
138       1.63      JP  1628106443  
207       0.47      TR  1628045921  
288       5.87      PT  1628060118  
311       0.61      AO  1628054683  
482       3.14      JP  1628107019  
484       4.63      BR  1628071047  
493       

In [104]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

In [105]:
# Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {
    "location": "",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [106]:
# Hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():
    params['location'] = f"{row['Lat']},{row['Lng']}"
    # params['keyword']
    response = requests.get(base_url, params=params).json()
    results = response['results']

# Store the first Hotel result into the DataFrame.
    try:
        print(f"The first hotel found in {row['City']} is {results[0]['name']}.")
              
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except(KeyError, IndexError):
        print("------------------")

The first hotel found in Pellérd is Hotel Laterum Pécs.
The first hotel found in Hasaki is Hotel Sunrise Choshi.
The first hotel found in Emirdağ is Emirdağ Öğretmenevi.
The first hotel found in Lagoa is Casa Das Faias.
The first hotel found in Benguela is Hotel Praia Morena.
The first hotel found in Shimoda is Shimoda Tokyu Hotel.
The first hotel found in Londrina is Hotel Crillon em Londrina.
The first hotel found in Vysokyy is ACAPULCO.


In [107]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

In [108]:
# Add marker layer on top of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))